In [ ]:
import pandas as pd
import json
import os
import time
import requests
import json
import pandas as pd


In [ ]:

input_path = '../dataset/collecte.json'
output_path = '../generated/normalized/collecte.json'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(input_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

for feature in data.get('features', []):
    props = feature.get('properties', {})
    if props.get('MUNICIPALITE') == 'Montreal':
        props['PROVINCE'] = 'Québec'

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)
print(f"Fichier modifié enregistré sous {output_path}")

In [ ]:

# Extraction des données du Québec uniquement
df = pd.read_csv('../dataset/Matieres_residuelles_recuperees.csv', sep=';')
df_quebec = df[df['GÉO'].str.contains('Québec', case=False, na=False)].copy()
print(f"Nombre de lignes pour le Québec : {len(df_quebec)}")
df_quebec.to_csv('../generated/normalized/matieres_residuelles_quebec.csv', index=False)
print("✅ Fichier 'matieres_residuelles_quebec.csv' généré avec succès.")

In [ ]:

input_path = '../dataset/matieres-residuelles-bilan-massique.csv'
output_path = '../generated/normalized/matieres_residuelles_massique.csv'
cache_file = '../cache/territoire_gps_cache.json'
os.makedirs(os.path.dirname(cache_file), exist_ok=True)

# Charger le cache existant si disponible
if os.path.exists(cache_file):
    with open(cache_file, 'r', encoding='utf-8') as f:
        cache = json.load(f)
else:
    cache = {}


# Charger le fichier massique
df = pd.read_csv(input_path)

def get_gps(territoire):
    if territoire in cache:
        return cache[territoire]
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': f"{territoire}, Québec, Canada",
        'format': 'json',
        'limit': 1
    }
    response = requests.get(url, params=params, headers={'User-Agent': 'geo-app'})
    if response.status_code == 200 and response.json():
        data = response.json()[0]
        lat, lon = data['lat'], data['lon']
        cache[territoire] = (lat, lon)
        time.sleep(1.1)  # Respecter la politique d'usage de Nominatim
        print(f"GPS pour {territoire} : {lat}, {lon}")
        return lat, lon
    return None, None

# Appliquer la fonction à chaque territoire unique
gps_coords = df['territoire'].dropna().unique()
territoire_gps = {t: get_gps(t) for t in gps_coords}

# Ajouter les colonnes latitude/longitude au DataFrame
df['latitude'] = df['territoire'].map(lambda t: territoire_gps.get(t, (None, None))[0])
df['longitude'] = df['territoire'].map(lambda t: territoire_gps.get(t, (None, None))[1])

# Après la boucle, sauvegarder le cache
with open(cache_file, 'w', encoding='utf-8') as f:
    json.dump(cache, f, ensure_ascii=False, indent=2)

# Sauvegarder le résultat
df.to_csv(output_path, index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/matieres-residuelles-bilan-massique.csv'

In [ ]:
# Normalisation des coordonnées GPS des centres de tri via OpenStreetMap Nominatim

input_path = '../dataset/liste-centres-tri-crd.csv'
output_path = '../generated/normalized/liste-centres-tri-crd.csv'
cache_file = '../cache/geocode_cache.csv'
df_sites = pd.read_csv(input_path)
df_sites['Code Postal '] = df_sites['Code Postal '].astype(str)
# On force les colonnes latitude/longitude à None si elles n'existent pas
if 'latitude' not in df_sites.columns:
    df_sites['latitude'] = None
if 'longitude' not in df_sites.columns:
    df_sites['longitude'] = None
# Charger le cache existant si disponible
os.makedirs(os.path.dirname(cache_file), exist_ok=True)
if os.path.exists(cache_file):
    with open(cache_file, 'r', encoding='utf-8') as f:
        geocode_cache = json.load(f)
else:
    geocode_cache = {}

def geocode_address(row):
    # Si déjà géocodé dans le DataFrame, ne rien faire
    if pd.notnull(row.get('latitude')) and pd.notnull(row.get('longitude')):
        return row['latitude'], row['longitude']
    address = f"{row['Adresse ']}, {row['Municipalité ']}, {row['Code Postal ']}, Québec, Canada"
    if address in geocode_cache:
        print(f"Cache hit for: {address}")
        return geocode_cache[address]
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': address,
        'format': 'json',
        'limit': 1,
    }
    response = requests.get(url, params=params, headers={'User-Agent': 'geo-app'})
    if response.status_code == 200 and response.json():
        data = response.json()[0]
        lat, lon = data['lat'], data['lon']
        geocode_cache[address] = (lat, lon)
        # Sauvegarder le cache à chaque nouvelle requête avec accents non échappés
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(geocode_cache, f, ensure_ascii=False, indent=2)
        return lat, lon
    return None, None

for i, row in df_sites.iterrows():
    lat, lon = geocode_address(row)
    df_sites.at[i, 'latitude'] = lat
    df_sites.at[i, 'longitude'] = lon
    print(f"{row['Adresse ']} → {lat}, {lon}")
    time.sleep(1.1)

df_sites.to_csv(output_path, index=False)
print("✅ Fichier 'liste-centres-tri-crd.csv' mis à jour avec succès.")